In [1]:
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
import random
import pandas as pd
import datetime
import pytz
import yaml
import threading
import time
from binance import Client, ThreadedWebsocketManager
from readAndSortCsv import read_and_sort_csv
from TradingStatisticsCalculator import TradingStatisticsCalculator
from HybridCNN import HybridCNN
from ta import trend, momentum, volatility, volume
import nest_asyncio

In [2]:
nest_asyncio.apply()

In [3]:
def load_config(config_path='config.yaml'):
    with open(config_path, 'r') as file:
        return yaml.safe_load(file)

config = load_config()

In [4]:
MODEL_PATH = 'simple1dcnn_state_dict.pth'
required_columns = ['date', 'open', 'high', 'low', 'close', 'volume']
file_path = r"DataFromBinance.csv"
input_window = 150
output_window = 15
np.set_printoptions(formatter={'float_kind': lambda x: f'{x:.2f}'})
random.seed(42)
np.random.seed(42)
client = Client(api_key, api_secret)
symbol = 'BTCUSDT'
interval = Client.KLINE_INTERVAL_1MINUTE


In [5]:
def get_dynamic_time_range(minutes_back=183):
    utc_now = datetime.datetime.utcnow().replace(tzinfo=pytz.UTC)
    end_time = utc_now
    start_time = end_time - datetime.timedelta(minutes=minutes_back)
    return start_time, end_time

In [6]:
start_time, end_time = get_dynamic_time_range()

C:\Users\Dima03\AppData\Local\Temp\ipykernel_16308\3472219458.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  utc_now = datetime.datetime.utcnow().replace(tzinfo=pytz.UTC)


In [7]:
try:
    klines = client.get_historical_klines(
        symbol=symbol,
        interval=interval,
        start_str=start_time.strftime("%d %b %Y %H:%M:%S"),
        end_str=end_time.strftime("%d %b %Y %H:%M:%S")
    )
except Exception as e:
    print(f"An error occurred while fetching klines: {e}")
    klines = []

if not klines:
    raise ValueError("No kline data was fetched. Please check your API credentials and time range.")


In [8]:
# Convert to DataFrame
df_M = pd.DataFrame(klines, columns=[
    'Open Time', 'Open', 'High', 'Low', 'Close', 'Volume',
    'Close Time', 'Quote Asset Volume', 'Number of Trades',
    'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'
])

# Convert columns to appropriate data types
columns_to_convert = [
    'Open', 'High', 'Low', 'Close', 'Volume',
    'Quote Asset Volume', 'Number of Trades',
    'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume'
]

In [9]:
for col in columns_to_convert:
    df_M[col] = df_M[col].astype(float)

# Format the DataFrame
def format_dataframe(df):
    df['date'] = pd.to_datetime(df['Open Time'], unit='ms')
    df_formatted = df[['date', 'Open', 'High', 'Low', 'Close', 'Volume']].copy()
    df_formatted.rename(columns={
        'Open': 'open',
        'High': 'high',
        'Low': 'low',
        'Close': 'close',
        'Volume': 'volume'
    }, inplace=True)
    df_formatted.set_index('date', inplace=True)
    return df_formatted

df_formatted = format_dataframe(df_M)

# Ensure we have the last 183 candles
df_formatted = df_formatted.tail(183).copy()


In [10]:
algo_calc = TradingStatisticsCalculator(
    initial_capital=5000.0,
    position_size_dollars=1000.0,
    close_idx=3,
    high_idx=1,
    low_idx=2,
    commission_rate=0.0005,
    tp_percent=0.0034,
    sl_percent=0.0033
)

In [11]:
def compute_features(df_input):
    df = df_input.iloc[:]
    df = df.iloc[::-1]
    df['SMA_20'] = df['close'].rolling(window=20).mean()
    df['EMA_20'] = df['close'].ewm(span=20, adjust=False).mean()

    df['RSI_14'] = momentum.RSIIndicator(close=df['close'], window=14).rsi()

    macd = trend.MACD(close=df['close'])
    df['MACD'] = macd.macd()
    df['MACD_Signal'] = macd.macd_signal()
    df['MACD_Diff'] = macd.macd_diff()

    bollinger = volatility.BollingerBands(close=df['close'], window=20, window_dev=2)
    df['Bollinger_High'] = bollinger.bollinger_hband()
    df['Bollinger_Low'] = bollinger.bollinger_lband()
    df['Bollinger_Middle'] = bollinger.bollinger_mavg()

    df['ATR_14'] = volatility.AverageTrueRange(high=df['high'], low=df['low'], close=df['close'], window=14).average_true_range()

    df['OBV'] = volume.OnBalanceVolumeIndicator(close=df['close'], volume=df['volume']).on_balance_volume()

    stochastic = momentum.StochasticOscillator(high=df['high'], low=df['low'], close=df['close'], window=14, smooth_window=3)
    df['Stochastic_%K'] = stochastic.stoch()
    df['Stochastic_%D'] = stochastic.stoch_signal()

    ichimoku = trend.IchimokuIndicator(high=df['high'], low=df['low'], window1=9, window2=26, window3=52)
    df['Ichimoku_A'] = ichimoku.ichimoku_a()
    df['Ichimoku_B'] = ichimoku.ichimoku_b()
    df['Ichimoku_Base_Line'] = ichimoku.ichimoku_base_line()
    df['Ichimoku_Conversion_Line'] = ichimoku.ichimoku_conversion_line()
    df.dropna(inplace=True)

    return df.iloc[::-1]

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = HybridCNN(num_features=22, seq_len=150, num_classes=3)

model.load_state_dict(torch.load(r"C:\GitCnn\CnnTrading\CnnTrans\HybridCNN_best_on_valid_state_dict.pth", map_location=device))

model.to(device)


C:\Users\Dima03\AppData\Local\Temp\ipykernel_16308\3530465443.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r"C:\GitCnn\CnnTrading\Cnn

HybridCNN(
  (shared_conv1): Conv1d(22, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (shared_relu): ReLU()
  (shared_dropout): Dropout1d(p=0.4, inplace=False)
  (feature_cnns): ModuleList(
    (0-21): 22 x Sequential(
      (0): Conv1d(1, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): ReLU()
      (2): Dropout1d(p=0.6, inplace=False)
      (3): Conv1d(128, 32, kernel_size=(3,), stride=(1,), padding=(1,))
      (4): ReLU()
      (5): Dropout1d(p=0.4, inplace=False)
    )
  )
  (fc1): Linear(in_features=110400, out_features=256, bias=True)
  (fc_dropout): Dropout(p=0.8, inplace=False)
  (fc2): Linear(in_features=256, out_features=3, bias=True)
)

In [13]:
class Predictor:
    def __init__(
        self,
        model):
        self.model = model
        self.model.eval()
    def predict(self, df_input):
        X_single_scaled = MinMaxScaler().fit_transform(df_input)  # shape (600,5)
        X_single_scaled = np.expand_dims(X_single_scaled, axis=0)  # => (1,600,5)
        X_single_transposed = np.transpose(X_single_scaled, (0, 2, 1))  # => (1,5,600)
        X_single_tensor = torch.from_numpy(X_single_transposed).float().to(device)
        with torch.no_grad():
            output = model(X_single_tensor)   # => shape (1,3)
            _, predicted = torch.max(output, 1)
            predicted_label = predicted.item()  # 0,1,2

        label_map = {0:"short",1:"flat",2:"long"}
        return label_map[predicted_label]

In [14]:
predictor = Predictor(model)

In [15]:
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class InputProvider:
    def __init__(self, initial_df, window_size=183, config_path='config.yaml'):
        self.window = window_size
        self.df = initial_df.copy()
        self.config_path = config_path
        self.lock = threading.Lock()
        self.new_candle = None
        self.ws_manager = ThreadedWebsocketManager(api_key=api_key, api_secret=api_secret)
        self.ws_manager.start()
        self.ws_manager.start_kline_socket(
            callback=self.handle_socket_message,
            symbol=symbol,
            interval=interval
        )
        self.running = True

    def handle_socket_message(self, msg):
        try:
            logger.info(msg)
            if msg['e'] != 'kline':
                return
            k = msg['k']
            if k['x']:  # If the candle is closed
                candle = {
                    'date': pd.to_datetime(k['t'], unit='ms'),
                    'open': float(k['o']),
                    'high': float(k['h']),
                    'low': float(k['l']),
                    'close': float(k['c']),
                    'volume': float(k['v'])
                }
                with self.lock:
                    self.df = self.df.append(candle, ignore_index=True)
                    self.df = self.df.iloc[-self.window:].copy()
                    self.new_candle = candle
        except Exception as e:
            logger.error(f"Error processing socket message: {e}")

    def can_get_next_input(self):
        # Reload config to check if we should continue
        config = load_config(self.config_path)
        if not config.get('run_prediction', False):
            self.running = False
            return False
        with self.lock:
            if self.new_candle is not None:
                return True
            else:
                return False

    def get_next_input(self):
        with self.lock:
            if self.new_candle is None:
                return None
            # Compute features on the updated DataFrame
            features_df = compute_features(self.df)
            self.new_candle = None
            return features_df.to_numpy()

    def stop(self):
        try:
            self.ws_manager.stop()
            logger.info("WebSocket connection stopped gracefully.")
        except Exception as e:
            logger.error(f"Error stopping WebSocket connection: {e}")


In [16]:
df_formatted

,open,high,low,close,volume
date,,,,,
2024-12-31 05:18:00,92400.19,92448.20,92400.19,92447.93,10.00467
2024-12-31 05:19:00,92447.93,92447.93,92415.48,92415.49,6.45410
2024-12-31 05:20:00,92415.48,92463.46,92405.63,92463.46,5.05557
2024-12-31 05:21:00,92463.46,92573.34,92463.45,92570.76,10.94399
2024-12-31 05:22:00,92571.20,92651.54,92571.20,92651.53,12.82284
...,...,...,...,...,...
2024-12-31 08:16:00,92992.02,93038.05,92983.63,93038.05,14.40258
2024-12-31 08:17:00,93038.05,93277.82,93038.05,93222.39,78.86181
2024-12-31 08:18:00,93222.39,93328.58,93213.61,93289.14,26.95198


In [17]:
input_provider = InputProvider(initial_df=df_formatted)

In [18]:
def run_prediction():
    try:
        while input_provider.running:
            if input_provider.can_get_next_input():
                df_input = input_provider.get_next_input()
                if df_input is not None:
                    predicted_label = predictor.predict(df_input)
                    latest_candle = input_provider.df.iloc[-1].to_dict()
                    algo_calc.process_candle(latest_candle, predicted_label)
            else:
                time.sleep(1)  # Wait before checking again
    except Exception as e:
        print(f"An error occurred in the prediction loop: {e}")
    finally:
        input_provider.stop()

# Start the prediction loop in a separate thread
prediction_thread = threading.Thread(target=run_prediction, daemon=True)
prediction_thread.start()

INFO:__main__:WebSocket connection stopped gracefully.


In [19]:
algo_stats = algo_calc.get_statistics()
algo_stats.to_dataframe().T

ERROR:asyncio:Exception in callback Task.__step()
handle: <Handle Task.__step()>
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2288.0_x64__qbz5n2kfra8p0\Lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2288.0_x64__qbz5n2kfra8p0\Lib\asyncio\tasks.py", line 305, in __step
    _leave_task(self._loop, self)
RuntimeError: Leaving task <Task pending name='Task-3' coro=<ThreadedApiManager.start_listener() running at c:\GitCnn\CnnTrading\CnnTrans\.venv\Lib\site-packages\binance\ws\threaded_stream.py:49> wait_for=<Future pending cb=[_chain_future.<locals>._call_check_cancel() at C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2288.0_x64__qbz5n2kfra8p0\Lib\asyncio\futures.py:389, Task.__wakeup()]>> does not match the current task <Task pending name='Task-2' coro=<Kernel.dispatch_queue()

,0
Initial Capital,5000.0
Final Capital,5000.0
Total Profit,0.0
Average Profit,0.0
Return on Investment (ROI),0.0
Number of Trades,0.0
Long Trades,0.0
Short Trades,0.0
Flat Trades,0.0
Position Size per Trade,1000.0
